In [2]:
import os
os.getcwd()
import sys 
sys.path.append('/data/users/miroojin/saksham/watermark-adapters')
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

In [4]:
import json
from src.mbmark import MbMark, Mode
from src.gaussmark import GaussMark
from sklearn.metrics import roc_auc_score
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datasets import load_dataset, load_from_disk
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy.stats
torch.manual_seed(42)
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.interpolate import interp1d


In [20]:
model_name = "mistralai/Mistral-7B-v0.3"
model_name = "meta-llama/Llama-2-7b-hf"
model_suffix = model_name.split("/")[-1]
output_file = "../output/Llama-2-7b-hf/output_align=0_delta=1.0_gamma=0.3_k=128_seed=15485863_watermark=mb_dataset=realnewslike.json"
output_file = "../output/Llama-2-7b-hf/output_seed=12997009_delta=1.0_watermark=mb2_dataset=realnewslike.json"
assert output_file is not None, "Please set the OUTPUT_FILE environment variable to the path of your output file."


BATCH_SIZE = 64
print(f"Batch size: {BATCH_SIZE}")
print(f"Model name: {model_name}")


Batch size: 64
Model name: meta-llama/Llama-2-7b-hf


In [17]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    device_map="auto")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", torch_dtype=torch.bfloat16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:

with open(output_file, "r") as f:
    output_data = json.load(f)

watermark_type = output_data["watermark"]
config = output_data["config"]

if watermark_type == "gaussmark":
    watermark = GaussMark(sigma=config["sigma"],
                          tokenizer=tokenizer, model=model, seed=config["hash_key"], target_param_name=config["target_param_name"])
    BATCH_SIZE = 8
elif watermark_type == "mb":
    final_weight = torch.tensor(output_data["final_matrix"])
    watermark = MbMark.mb(
        delta=config["delta"],
        gamma=config["gamma"],
        seed=config["hash_key"],
        final_weight=final_weight,
        model=model,
        tokenizer=tokenizer,
        unembedding_param_name=config["unembedding_param_name"],
        mode=Mode.Generate,
    )
elif watermark_type == "mb2":
    watermark = MbMark.mb2(
        delta=config["delta"],
        seed=config["hash_key"],
        model=model,
        tokenizer=tokenizer,
        unembedding_param_name=config["unembedding_param_name"],
        mode=Mode.Generate
    )

In [22]:
watermarked_model = watermark.model

if watermark_type == "mb":
    watermarked_model_name = f"{model_name}-watermarked-greenlist-bias-k128-seed{config['hash_key']}"
elif watermark_type == "mb2":
    watermarked_model_name = f"{model_name}-watermarked-arcsine-noise-seed{config['hash_key']}"

print(f"Watermarked model name: {watermarked_model_name}")

# Save the model to /pool.ssd/assets/models/
watermarked_model_name = os.path.join(
    "/pool.ssd/assets/models/", watermarked_model_name)
if not os.path.exists(watermarked_model_name):
    os.makedirs(watermarked_model_name)
# Save the model to the specified directory
watermarked_model.save_pretrained(watermarked_model_name)
tokenizer.save_pretrained(watermarked_model_name)
print(f"Watermarked model saved to {watermarked_model_name}")

Watermarked model name: meta-llama/Llama-2-7b-hf-watermarked-arcsine-noise-seed12997009
Watermarked model saved to /pool.ssd/assets/models/meta-llama/Llama-2-7b-hf-watermarked-arcsine-noise-seed12997009
